# Accessing the cloud with fsspec

> Filesystem Spec (fsspec) is a project to provide a unified pythonic interface to local, remote and embedded file systems and bytes storage. fsspec provides two main concepts: a set of filesystem classes with uniform APIs (i.e., functions such as cp, rm, cat, mkdir, …) supplying operations on a range of storage systems; and top-level convenience functions like fsspec.open(), to allow you to quickly get from a URL to a file-like object that you can use with a third-party library or your own code.

This notebook introduces

- the fsspec file system object
- the fsspec mapper
- url chaining

to create, copy and delete objects within DRKZ´s insitutional cloud and other storages and use it in xarray.

## Preparation

### 1. Create access token and load them

In [ ]:
%%bash
module load py-python-swiftclient/3.12.0-gcc-11.2.0
swift-token new
#Account: bk1377
#Username: YOUR-USER-NAME
module switch py-python-swiftclient
#cat ~/.swiftenv
#write these tokens into a token.py file that can be loaded from a python script

In [1]:
from mytoken import *

### 2. Set up environment in order to work with fsspec

In [2]:
import os
import fsspec
os.environ["OS_STORAGE_URL"]=OS_STORAGE_URL
os.environ["OS_AUTH_TOKEN"]=OS_AUTH_TOKEN

## The File system object

to use basic shell commands on different storage back ends

In [3]:
fsswift=fsspec.filesystem("swift")

What is stored under the account?

In [4]:
fsswift.ls(OS_STORAGE_URL)

[{'name': 'swift://swift.dkrz.de/dkrz_5cfad75f-8778-40d2-bdc0-eec8ae27ad1f/dkrz_scratch',
  'size': 0,
  'type': 'directory'},
 {'name': 'swift://swift.dkrz.de/dkrz_5cfad75f-8778-40d2-bdc0-eec8ae27ad1f/hackathon2023-results',
  'size': 25961183,
  'type': 'directory'},
 {'name': 'swift://swift.dkrz.de/dkrz_5cfad75f-8778-40d2-bdc0-eec8ae27ad1f/k204210',
  'size': 122003733,
  'type': 'directory'}]

Note that a swift path starts with *swift://*. The OS_STORAGE_URL is internally translated, we could it as well:

In [5]:
swift_account_url=OS_STORAGE_URL.replace("https://","swift://").replace("/v1/","/")
fsswift.ls(swift_account_url)

[{'name': 'swift://swift.dkrz.de/dkrz_5cfad75f-8778-40d2-bdc0-eec8ae27ad1f/dkrz_scratch',
  'size': 0,
  'type': 'directory'},
 {'name': 'swift://swift.dkrz.de/dkrz_5cfad75f-8778-40d2-bdc0-eec8ae27ad1f/hackathon2023-results',
  'size': 25961183,
  'type': 'directory'},
 {'name': 'swift://swift.dkrz.de/dkrz_5cfad75f-8778-40d2-bdc0-eec8ae27ad1f/k204210',
  'size': 122003733,
  'type': 'directory'}]

How big is a container inside the storage?

In [6]:
fsswift.du(OS_STORAGE_URL+"/hackathon2023-results")

25961183

Note: Container cannot be created with fsspec. Use the GUI for that.

Note: There are no directories under a container BUT fsspec lets you work with the path as if there would be:

In [7]:
fsswift.mkdir(OS_STORAGE_URL+"/k204210/VIRTUAL_DIRECTORY")

Create objects/files:

In [8]:
with fsswift.open(OS_STORAGE_URL+"/k204210/VIRTUAL_DIRECTORY/newtestfile","w") as f:
    f.write("My first file in the cloud")

Delete files:

In [9]:
fsswift.rm(OS_STORAGE_URL+"/k204210/VIRTUAL_DIRECTORY/newtestfile")

[None]

Note that "mv" in a cloud does not exist. It is an expensive copy and remove. So if you put data in the cloud,

- the location should be fix (otherwise it is a mv again)
- the data should be fix (update also means delete and recreate)

## The mapper

can create a dict-like object for any url:

In [10]:
cloudmapper=fsswift.get_mapper(OS_STORAGE_URL+"/k204210")

Has an underlying file system again:

In [11]:
cloudmapper.fs

Detects the protocol by parsing the first characters of the path:

In [12]:
diskmapper=fsspec.get_mapper("/work/ik1017/CMIP6/data/CMIP6/ScenarioMIP/DKRZ/MPI-ESM1-2-HR/ssp126/r1i1p1f1/Amon/tas/gn/v20190710/")

In [13]:
diskmapper.fs

Is a key-value storage: All keys of the dict correspond to files or object ids whereas the values of each key is the byte content of the key.

In [14]:
print(list(diskmapper.keys()))

['tas_Amon_MPI-ESM1-2-HR_ssp126_r1i1p1f1_gn_201501-201912.nc', 'tas_Amon_MPI-ESM1-2-HR_ssp126_r1i1p1f1_gn_202001-202412.nc', 'tas_Amon_MPI-ESM1-2-HR_ssp126_r1i1p1f1_gn_202501-202912.nc', 'tas_Amon_MPI-ESM1-2-HR_ssp126_r1i1p1f1_gn_203001-203412.nc', 'tas_Amon_MPI-ESM1-2-HR_ssp126_r1i1p1f1_gn_203501-203912.nc', 'tas_Amon_MPI-ESM1-2-HR_ssp126_r1i1p1f1_gn_204001-204412.nc', 'tas_Amon_MPI-ESM1-2-HR_ssp126_r1i1p1f1_gn_204501-204912.nc', 'tas_Amon_MPI-ESM1-2-HR_ssp126_r1i1p1f1_gn_205001-205412.nc', 'tas_Amon_MPI-ESM1-2-HR_ssp126_r1i1p1f1_gn_205501-205912.nc', 'tas_Amon_MPI-ESM1-2-HR_ssp126_r1i1p1f1_gn_206001-206412.nc', 'tas_Amon_MPI-ESM1-2-HR_ssp126_r1i1p1f1_gn_206501-206912.nc', 'tas_Amon_MPI-ESM1-2-HR_ssp126_r1i1p1f1_gn_207001-207412.nc', 'tas_Amon_MPI-ESM1-2-HR_ssp126_r1i1p1f1_gn_207501-207912.nc', 'tas_Amon_MPI-ESM1-2-HR_ssp126_r1i1p1f1_gn_208001-208412.nc', 'tas_Amon_MPI-ESM1-2-HR_ssp126_r1i1p1f1_gn_208501-208912.nc', 'tas_Amon_MPI-ESM1-2-HR_ssp126_r1i1p1f1_gn_209001-209412.nc', 'tas_Am

Can be used to copy files. Be careful: the keys in the mapper are for all subdirectories!

In [15]:
cloudmapper.update(diskmapper)

Saves the underlying root path:

In [16]:
fsswift.ls(cloudmapper.root)

[{'name': 'swift://swift.dkrz.de/dkrz_5cfad75f-8778-40d2-bdc0-eec8ae27ad1f/k204210/tas_Amon_MPI-ESM1-2-HR_ssp126_r1i1p1f1_gn_201501-201912.nc',
  'size': 7094228,
  'type': 'file',
  'last_modified': '2023-12-05T09:19:01.705300',
  'hash': '4093945c24f7e07fe99a63b6de534d14',
  'content_type': 'application/octet-stream'},
 {'name': 'swift://swift.dkrz.de/dkrz_5cfad75f-8778-40d2-bdc0-eec8ae27ad1f/k204210/tas_Amon_MPI-ESM1-2-HR_ssp126_r1i1p1f1_gn_202001-202412.nc',
  'size': 7092506,
  'type': 'file',
  'last_modified': '2023-12-05T09:19:04.438440',
  'hash': '1988ff02cc430b6046255993e4b1e6b0',
  'content_type': 'application/octet-stream'},
 {'name': 'swift://swift.dkrz.de/dkrz_5cfad75f-8778-40d2-bdc0-eec8ae27ad1f/k204210/tas_Amon_MPI-ESM1-2-HR_ssp126_r1i1p1f1_gn_202501-202912.nc',
  'size': 7092539,
  'type': 'file',
  'last_modified': '2023-12-05T09:19:04.708350',
  'hash': '6469e165a86f07645b933e613de99e32',
  'content_type': 'application/octet-stream'},
 {'name': 'swift://swift.dkrz.d

fsspec open files can be used in xarray:

In [17]:
files_for_xarray=[
    fsspec.open(a["name"]).open()
    for a in fsswift.ls(cloudmapper.root)
]

In [18]:
import xarray as xr
xr.open_mfdataset(files_for_xarray)

<xarray.Dataset>
Dimensions:    (time: 1032, bnds: 2, lat: 192, lon: 384)
Coordinates:
  * time       (time) datetime64[ns] 2015-01-16T12:00:00 ... 2100-12-16T12:00:00
  * lat        (lat) float64 -89.28 -88.36 -87.42 -86.49 ... 87.42 88.36 89.28
  * lon        (lon) float64 0.0 0.9375 1.875 2.812 ... 356.2 357.2 358.1 359.1
    height     float64 2.0
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] dask.array<chunksize=(1, 2), meta=np.ndarray>
    lat_bnds   (time, lat, bnds) float64 dask.array<chunksize=(60, 192, 2), meta=np.ndarray>
    lon_bnds   (time, lon, bnds) float64 dask.array<chunksize=(60, 384, 2), meta=np.ndarray>
    tas        (time, lat, lon) float32 dask.array<chunksize=(1, 192, 384), meta=np.ndarray>
Attributes: (12/47)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            ScenarioMIP
    branch_method:          standard
    branch_time_in_child:   60265.0
    branch_time_in_parent:  60265.0
    contact:                cmip6-mpi-esm@dkrz.de
    ...                     ...
    title:                  MPI-ESM1-2-HR output prepared for CMIP6
    variable_id:            tas
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by DKRZ is licensed und...
    cmor_version:           3.4.0
    tracking_id:            hdl:21.14100/981bf19d-e45d-478d-97a8-3214485079b6

## URL chains

URLs can be chained with the separator `::`. In this example, we access a specific file in a zip container.

In [19]:
!zip /home/k/k204210/eerie-io/analysis_support/temp.zip /work/ik1017/CMIP6/data/CMIP6/ScenarioMIP/DKRZ/MPI-ESM1-2-HR/ssp126/r1i1p1f1/Amon/tas/gn/v20190710/*

updating: work/ik1017/CMIP6/data/CMIP6/ScenarioMIP/DKRZ/MPI-ESM1-2-HR/ssp126/r1i1p1f1/Amon/tas/gn/v20190710/tas_Amon_MPI-ESM1-2-HR_ssp126_r1i1p1f1_gn_201501-201912.nc (deflated 1%)
updating: work/ik1017/CMIP6/data/CMIP6/ScenarioMIP/DKRZ/MPI-ESM1-2-HR/ssp126/r1i1p1f1/Amon/tas/gn/v20190710/tas_Amon_MPI-ESM1-2-HR_ssp126_r1i1p1f1_gn_202001-202412.nc (deflated 1%)
updating: work/ik1017/CMIP6/data/CMIP6/ScenarioMIP/DKRZ/MPI-ESM1-2-HR/ssp126/r1i1p1f1/Amon/tas/gn/v20190710/tas_Amon_MPI-ESM1-2-HR_ssp126_r1i1p1f1_gn_202501-202912.nc (deflated 1%)
updating: work/ik1017/CMIP6/data/CMIP6/ScenarioMIP/DKRZ/MPI-ESM1-2-HR/ssp126/r1i1p1f1/Amon/tas/gn/v20190710/tas_Amon_MPI-ESM1-2-HR_ssp126_r1i1p1f1_gn_203001-203412.nc (deflated 1%)
updating: work/ik1017/CMIP6/data/CMIP6/ScenarioMIP/DKRZ/MPI-ESM1-2-HR/ssp126/r1i1p1f1/Amon/tas/gn/v20190710/tas_Amon_MPI-ESM1-2-HR_ssp126_r1i1p1f1_gn_203501-203912.nc (deflated 1%)
updating: work/ik1017/CMIP6/data/CMIP6/ScenarioMIP/DKRZ/MPI-ESM1-2-HR/ssp126/r1i1p1f1/Amon/tas/

In [20]:
xr.open_dataset(
    fsspec.open(
        'zip://work/ik1017/CMIP6/data/CMIP6/ScenarioMIP/DKRZ/MPI-ESM1-2-HR/ssp126/r1i1p1f1/Amon/tas/gn/v20190710/tas_Amon_MPI-ESM1-2-HR_ssp126_r1i1p1f1_gn_201501-201912.nc'+
        "::"+
        "/home/k/k204210/eerie-io/analysis_support/temp.zip"
    ).open()
)

<xarray.Dataset>
Dimensions:    (time: 60, bnds: 2, lat: 192, lon: 384)
Coordinates:
  * time       (time) datetime64[ns] 2015-01-16T12:00:00 ... 2019-12-16T12:00:00
  * lat        (lat) float64 -89.28 -88.36 -87.42 -86.49 ... 87.42 88.36 89.28
  * lon        (lon) float64 0.0 0.9375 1.875 2.812 ... 356.2 357.2 358.1 359.1
    height     float64 ...
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) datetime64[ns] ...
    lat_bnds   (lat, bnds) float64 ...
    lon_bnds   (lon, bnds) float64 ...
    tas        (time, lat, lon) float32 ...
Attributes: (12/47)
    Conventions:            CF-1.7 CMIP-6.2
    activity_id:            ScenarioMIP
    branch_method:          standard
    branch_time_in_child:   60265.0
    branch_time_in_parent:  60265.0
    contact:                cmip6-mpi-esm@dkrz.de
    ...                     ...
    title:                  MPI-ESM1-2-HR output prepared for CMIP6
    variable_id:            tas
    variant_label:          r1i1p1f1
    license:                CMIP6 model data produced by DKRZ is licensed und...
    cmor_version:           3.4.0
    tracking_id:            hdl:21.14100/981bf19d-e45d-478d-97a8-3214485079b6

This allows us to e.g. access specific files from a tar archive.

There are many [implementations](https://filesystem-spec.readthedocs.io/en/latest/api.html#built-in-implementations). for fsspec.                                 